## Grouping all the data for the test without initial global search
The search is started from the initial solution defined as the best solution found from the initial global search performed over the whole parameter space:
$$\theta_{init}^T = \theta_{argmin(\ell(Y_T, Y_{\theta_{\epsilon}}))} \in \theta_{\epsilon} \; \forall \; T\in \text{Set of 20 targets}$$

### Imports & settings

In [1]:
import numpy as np
import torch
from matplotlib import pyplot as plt
import matplotlib as mpl
import pickle as pkl
from lib.utils import *
from copy import deepcopy as dcp
from os import walk
import shutil
from lib.Wilson_Cowan.parameters_info import parameter_names, parameters_lower_bound,parameters_range_bounds
mpl.rcParams['font.size'] = 18
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.facecolor'] = 'w'

### Loading the data from the CMA-ES search

In [ ]:
cma_p = '../Data/WC/search_tests/cmaes/'
target_indices = [i for i in range(20)]
results = [[None for _ in range(8)] for _ in range(2)]
runtime_results = [[None for _ in range(8)] for _ in range(2)]
num_std_tested = [1]
init_files = []
runtime_CMA = [[] for _ in range(2)]
all_std_errors = [[] for _ in range(2)]
cma_fs = ['cmaes_25chk_100rd_all_best_final25repeats_0']
for iii, cma_f in enumerate(cma_fs):
    cmaes_log = get_log(cma_p,cma_f)
    for ii in range(2):
        start_index = ii * 20
        init_file_to_search = cmaes_log[start_index]['settings']['init_file']
        print(init_file_to_search)
        init_files.append(init_file_to_search)
        for j in range(num_std_tested[iii]):
            cmaes_errors = []
            run_time = []
            sigma = cmaes_log[start_index+j]['settings']['sigma0']
            for i in range(20):
                errors = -cmaes_log[start_index+i]['fit_history']['error']
                run_time.append(cmaes_log[start_index+i]['overall_runtime'])
                cmaes_errors.append(errors)
            cmaes_errors = torch.cat(cmaes_errors, dim = 0).mean(-1).unsqueeze(0)
            cmaes_fits = torch.cat(cmaes_fits, dim = 0)
            results[ii][3] = cmaes_errors
            run_time = torch.cat(run_time, dim = 0).mean(-1)
            runtime_results[ii][3] = run_time.unsqueeze(0)
print(results[0][3].shape)
print(runtime_results[0][3].shape)

### Loading the data from the PSO search

In [3]:
global_search_path_PSO = '../Data/WC/search_tests/PSO/'
PSO_paths = [
    'cube_10000_0-20targets_distsphere_blw0.15_chunk25_100rds_w0.7_c2.25_c2.25_PSO_0',
    'cube_10000_1-20targets_distsphere_blw0.15_chunk25_100rds_w0.9_c1.5_c1.5_PSO_0',
]
search_logs = [get_log(global_search_path_PSO, PSO_paths[i]) for i in range(len(PSO_paths))]
for i in range(len(PSO_paths)):
    results[i][2] = -search_logs[i][0]['fit_history']['error'][target_indices].mean(-1).unsqueeze(0)
    runtime_results[i][2] = search_logs[i][0]['overall_runtime'].squeeze().unsqueeze(0)
print(results[0][2].shape)
print(runtime_results[0][2].shape)

### Loading the data from the RS & SA searches (baseline and DR-FFIT augmented versions)

In [6]:
global_search_path = '../Data/WC/search_tests/simple/'
final_files = [
        ['cube_10000_0-20targets_w0.0_blw0.125_chunk25_100rds_0dim_5LD_pure', 0],
		['cube_10000_0-20targets_w0.0_blw0.1_chunk25_100rds_0dim_5LD_SA', 1],
		['cube_10000_0-20targets_w0.15_blw0.0_chunk25_100rds_8dim_5LD_pure_AE', 2],
		['cube_10000_0-20targets_w0.15_blw0.0_chunk25_100rds_10dim_5LD_pure_PCA', 3],
		['cube_10000_0-20targets_w0.15_blw0.0_chunk25_100rds_11dim_5LD_SA_AE', 4],
		['cube_10000_0-20targets_w0.15_blw0.0_chunk25_100rds_10dim_5LD_SA_PCA', 5],
],[
		['cube_10000_1-20targets_w0.0_blw0.1_chunk25_100rds_0dim_5LD_pure', 0],
		['cube_10000_1-20targets_w0.0_blw0.125_chunk25_100rds_0dim_5LD_SA', 1],
		['cube_10000_1-20targets_w0.15_blw0.0_chunk25_100rds_8dim_5LD_pure_AE', 2],
		['cube_10000_1-20targets_w0.15_blw0.0_chunk25_100rds_10dim_5LD_pure_PCA', 3],
		['cube_10000_1-20targets_w0.175_blw0.0_chunk25_100rds_11dim_5LD_SA_AE', 4],
		['cube_10000_1-20targets_w0.15_blw0.0_chunk25_100rds_12dim_5LD_SA_PCA', 5],
]

res_index = [0,1,4,5,6,7]
random_search_paths = final_files
for ii in range(len(random_search_paths)):
    fits_rs = [None for _ in range(6)]
    settings = [None for _ in range(6)]
    for j, path in enumerate(random_search_paths[ii]):
        file_er = get_log(global_search_path, path[0])
        err = -file_er[path[1]]['fit_history']['error'][target_indices][:,:,:101,:].mean(-1).unsqueeze(0)
        results[ii][res_index[path[1]]] = err
        runtime_results[ii][res_index[path[1]]] = file_er[path[1]]['overall_runtime'].squeeze().unsqueeze(0)
print(results[0][7].shape)
print(runtime_results[0][7].shape)

### Concatenating all the data

In [ ]:
results[0] = torch.cat(results[0], dim = 0).unsqueeze(0)
results[1] = torch.cat(results[1], dim = 0).unsqueeze(0)
results = torch.cat(results, dim = 0)
print(results.shape)
runtime_results[0] = torch.cat(runtime_results[0], dim = 0).unsqueeze(0)
runtime_results[1] = torch.cat(runtime_results[1], dim = 0).unsqueeze(0)
runtime_results = torch.cat(runtime_results, dim = 0)
print(runtime_results.shape)

### Saving the file to be loaded in the WC_analyze_data.ipynb file

In [ ]:
combined_data_path = "../Data/"
results_log = {
    'results': results,
    'runtime': runtime_results
}
save_log(results_log, combined_data_path, 'WC_with_init')